# Parameter optimization

In [3]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'buy'
pattern = ['Pattern', 'Trend']
indicator_list = pattern
indicator_list_higher = pattern

work_timeframe = '15m'
higher_timeframe = '1h'
opt_limit = 2000
load = False

print(f'Timeframe is {higher_timeframe}, trade type is {ttype}')

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Indicator_list'] = pattern
configs['Higher_TF_indicator_list'] = indicator_list_higher
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {
                'Pattern': {
                            'vol_q_high': [0.9], # np.arange(0.1, 1, 0.05),
                            'vol_q_low': [0.2], # np.arange(0.1, 1, 0.05),
                            'window_low_bound': [1, 2, 3], 
                            'window_high_bound': [4, 5, 6, 7],
                            'first_candle': [0.8], # np.arange(0.5, 1, 0.05),
                            'second_candle': [0.5], # np.arange(0.5, 1, 0.05)
                            },
                'Trend': {
                            'timeperiod': [16], # np.arange(2, 50, 2),
                            'low_bound': [0]
                            }
             }

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

# stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
# if not stat_list:
#     stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
# else:
#     stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')


Timeframe is 1h, trade type is buy
Number of combinations is 12


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [02:09<00:00, 10.77s/it]


# Check local statistics

In [3]:
# candles
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('e_ratio_avg', ascending=False)
stat.head(20)

,pattern,Pattern_vol_q_high,Pattern_vol_q_low,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Trend_timeperiod,Trend_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
79,Pattern_Trend,0.7,0.2,1,5,0.85,0.95,20,0,44,1.742033,0.278750,32.649467,12.265000,1.4261,1.5621,1.5959,1.2713,1.2145,1.8968,1.7955,1.8097,1.7648,1.7130,1.6819,1.6540,1.6404,1.8196,1.9325,1.9408,1.9822,1.9149,1.9196,1.9137,1.8420,1.8789,1.8676,1.7710,0.03,0.06,0.10,0.12,0.13,0.14,0.14,0.14,0.14,0.12,0.10,0.13,0.16,0.20,0.24,0.29,0.36,0.38,0.42,0.62,0.64,0.66,0.68,0.69
69,Pattern_Trend,0.7,0.2,1,5,0.80,0.95,20,0,49,1.667913,0.110417,32.727713,5.410417,1.4160,1.5553,1.6057,1.2866,1.2206,1.8375,1.7481,1.7280,1.6955,1.6448,1.6182,1.5656,1.5553,1.7145,1.8104,1.8215,1.8613,1.8043,1.8103,1.8032,1.7422,1.7704,1.7475,1.6671,0.01,0.02,0.05,0.07,0.07,0.07,0.06,0.06,0.05,0.02,-0.00,0.01,0.01,0.03,0.04,0.07,0.12,0.12,0.14,0.30,0.32,0.33,0.34,0.34
59,Pattern_Trend,0.7,0.2,1,5,0.75,0.95,20,0,59,1.604142,0.051250,35.644358,3.023750,1.3637,1.5077,1.4950,1.2328,1.1729,1.7388,1.6729,1.7534,1.7057,1.6183,1.5790,1.5305,1.5232,1.6651,1.7520,1.7427,1.7720,1.7097,1.7014,1.6967,1.6474,1.6712,1.6585,1.5888,0.01,0.03,0.05,0.07,0.06,0.06,0.04,0.05,0.04,0.02,-0.01,-0.00,-0.00,-0.00,0.00,0.01,0.03,0.01,0.02,0.15,0.15,0.15,0.15,0.14
89,Pattern_Trend,0.7,0.2,1,5,0.90,0.95,20,0,38,1.577796,0.159583,21.956242,6.064167,1.1605,1.2481,1.4262,1.1474,1.0824,1.8541,1.7227,1.7406,1.6950,1.6563,1.6992,1.6093,1.5632,1.7170,1.7639,1.7405,1.7557,1.6640,1.6711,1.6692,1.5861,1.6033,1.5926,1.4987,0.04,0.07,0.12,0.15,0.16,0.17,0.17,0.18,0.17,0.15,0.14,0.16,0.15,0.15,0.12,0.12,0.13,0.11,0.12,0.29,0.28,0.25,0.23,0.20
9,Pattern_Trend,0.7,0.2,1,5,0.50,0.95,20,0,100,1.529163,-0.089583,52.916250,-8.958333,1.2281,1.1906,1.1866,1.0801,1.0274,1.3954,1.3785,1.4386,1.4433,1.4394,1.4220,1.4128,1.4437,1.6715,1.8638,1.8325,1.8372,1.8004,1.8014,1.7982,1.7616,1.7676,1.7624,1.7168,-0.02,-0.03,-0.05,-0.07,-0.09,-0.12,-0.14,-0.15,-0.17,-0.19,-0.22,-0.24,-0.24,-0.21,-0.19,-0.17,-0.15,-0.13,-0.10,0.00,0.12,0.13,0.14,0.14
78,Pattern_Trend,0.7,0.2,1,5,0.85,0.90,20,0,56,1.528771,0.162500,29.611167,9.100000,1.4551,1.5738,1.5319,1.2564,1.2039,1.7487,1.6465,1.6808,1.5251,1.4673,1.4257,1.4206,1.4195,1.5582,1.6446,1.6275,1.6000,1.5727,1.5742,1.5844,1.5429,1.5714,1.5597,1.4996,0.04,0.08,0.12,0.15,0.16,0.17,0.16,0.16,0.14,0.10,0.08,0.09,0.09,0.08,0.09,0.10,0.11,0.10,0.11,0.16,0.41,0.41,0.40,0.39
49,Pattern_Trend,0.7,0.2,1,5,0.70,0.95,20,0,71,1.507150,-0.019167,36.007650,-1.360833,1.2090,1.2234,1.2980,1.1316,1.0904,1.5876,1.5653,1.6421,1.5911,1.5325,1.4923,1.4704,1.4567,1.5781,1.6449,1.6564,1.6805,1.6307,1.6554,1.6417,1.5989,1.6164,1.6173,1.5609,0.00,-0.00,0.01,0.02,0.01,0.00,-0.01,0.00,0.00,-0.00,-0.03,-0.03,-0.04,-0.05,-0.06,-0.06,-0.06,-0.09,-0.09,0.01,0.00,0.00,0.01,-0.00
90,Pattern_Trend,0.7,0.2,1,5,0.95,0.50,20,0,105,1.487692,0.676667,51.207625,71.050000,1.2168,1.2867,1.3244,1.2346,1.6213,1.5484,1.5378,1.5745,1.4709,1.4221,1.4059,1.3973,1.4232,1.4816,1.5159,1.5453,1.5533,1.5530,1.5803,1.6025,1.5412,1.5880,1.6556,1.6240,0.09,0.17,0.26,0.33,0.40,0.46,0.52,0.58,0.63,0.65,0.70,0.73,0.75,0.78,0.81,0.85,0.88,0.90,0.98,1.02,1.12,0.85,0.88,0.90
60,Pattern_Trend,0.7,0.2,1,5,0.80,0.50,20,0,179,1.485533,0.391250,86.9104

In [11]:
# LR timeperiod
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg', ascending=False)
stat.head(20)

,pattern,Pattern_vol_q_high,Pattern_vol_q_low,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Trend_timeperiod,Trend_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
7,Pattern_Trend,0.9,0.2,1,5,0.8,0.5,16,0,164,1.465504,0.392083,76.342683,64.301667,1.3958,1.4252,1.4184,1.3309,1.3317,1.5419,1.5118,1.5822,1.4609,1.3780,1.3338,1.3434,1.3452,1.4006,1.4464,1.4665,1.5002,1.5029,1.5434,1.5642,1.5420,1.5738,1.6249,1.6080,0.06,0.11,0.17,0.20,0.25,0.30,0.34,0.39,0.42,0.43,0.43,0.44,0.44,0.44,0.45,0.46,0.47,0.46,0.48,0.51,0.64,0.48,0.51,0.53
8,Pattern_Trend,0.9,0.2,1,5,0.8,0.5,18,0,162,1.473383,0.371250,76.688100,60.142500,1.3916,1.4231,1.4204,1.3444,1.3511,1.5658,1.5388,1.6066,1.4710,1.3819,1.3434,1.3501,1.3510,1.4052,1.4513,1.4722,1.5074,1.5086,1.5480,1.5658,1.5448,1.5770,1.6295,1.6122,0.06,0.10,0.15,0.18,0.23,0.27,0.32,0.37,0.40,0.40,0.40,0.41,0.41,0.42,0.43,0.44,0.45,0.44,0.46,0.49,0.62,0.46,0.49,0.51
6,Pattern_Trend,0.9,0.2,1,5,0.8,0.5,14,0,160,1.440304,0.368750,70.448667,59.000000,1.4179,1.4413,1.4298,1.3431,1.3400,1.5542,1.5207,1.5891,1.4634,1.3732,1.3276,1.3337,1.3196,1.3677,1.4052,1.4101,1.4325,1.4375,1.4776,1.4996,1.4770,1.5057,1.5579,1.5429,0.06,0.12,0.18,0.21,0.27,0.31,0.36,0.41,0.44,0.44,0.44,0.45,0.44,0.43,0.43,0.43,0.42,0.40,0.41,0.43,0.56,0.38,0.41,0.42
9,Pattern_Trend,0.9,0.2,1,5,0.8,0.5,20,0,162,1.482317,0.361250,78.135300,58.522500,1.3816,1.4350,1.4344,1.3536,1.3596,1.5750,1.5448,1.6125,1.4761,1.3882,1.3463,1.3528,1.3635,1.4175,1.4641,1.4845,1.5196,1.5193,1.5584,1.5781,1.5567,1.5890,1.6414,1.6236,0.06,0.10,0.15,0.18,0.23,0.27,0.32,0.36,0.39,0.39,0.39,0.40,0.40,0.40,0.41,0.42,0.43,0.43,0.44,0.47,0.61,0.44,0.48,0.50
3,Pattern_Trend,0.9,0.2,1,5,0.8,0.5,8,0,146,1.427479,0.349583,62.411958,51.039167,1.3604,1.3951,1.3706,1.2819,1.2497,1.4845,1.4315,1.4811,1.3827,1.3354,1.3128,1.3674,1.3487,1.3869,1.4197,1.4413,1.4687,1.4745,1.5120,1.5329,1.5074,1.5301,1.5914,1.5928,0.06,0.11,0.17,0.20,0.24,0.28,0.32,0.36,0.38,0.38,0.38,0.39,0.38,0.38,0.38,0.39,0.39,0.39,0.40,0.43,0.60,0.42,0.46,0.50
5,Pattern_Trend,0.9,0.2,1,5,0.8,0.5,12,0,153,1.441217,0.337917,67.506150,51.701250,1.3985,1.4535,1.4550,1.3548,1.3553,1.5645,1.5122,1.5779,1.4362,1.3448,1.3009,1.3231,1.3163,1.3660,1.4005,1.4115,1.4388,1.4439,1.4832,1.5079,1.4850,1.5154,1.5713,1.5727,0.06,0.11,0.17,0.21,0.26,0.30,0.35,0.39,0.42,0.42,0.41,0.42,0.41,0.40,0.39,0.38,0.37,0.35,0.35,0.38,0.51,0.32,0.35,0.38
12,Pattern_Trend,0.9,0.2,1,5,0.8,0.5,26,0,165,1.448021,0.332500,73.923438,54.862500,1.3830,1.3487,1.3483,1.2835,1.2970,1.5005,1.4813,1.5331,1.4247,1.3477,1.3080,1.3410,1.3555,1.4047,1.4430,1.4643,1.4951,1.5082,1.5450,1.5497,1.5486,1.5832,1.6306,1.6278,0.06,0.10,0.15,0.18,0.22,0.26,0.30,0.33,0.35,0.35,0.35,0.36,0.36,0.36,0.37,0.38,0.39,0.39,0.40,0.43,0.57,0.41,0.44,0.47
4,Pattern_Trend,0.9,0.2,1,5,0.8,0.5,10,0,150,1.483254,0.330000,72.488125,49.500000,1.3456,1.3942,1.3960,1.3022,1.2961,1.5249,1.4789,1.5526,1.4364,1.3876,1.3498,1.3990,1.3982,1.4610,1.4972,1.5172,1.5474,1.5495,1.5894,1.6129,1.5859,1.6178,1.6791,1.6792,0.05,0.10,0.15,0.18,0.22,0.26,0.30,0.35,0.37,0.37,0.37,0.37,0.37,0.36,0.36,0.36,0.36,0.36,0.37,0.41,0.57,0.39,0.44,0.48
11,Pattern_Trend,0.9,0.2,1,5,0.8,0.5,24,0,163,1.437975,0.330000,71.389925,53.790000,1.3554,1.3261,1.3230,1.2616,1.2765,1.4923

In [13]:
# vol
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg', ascending=False)
stat.head(20)

,pattern,Pattern_vol_q_high,Pattern_vol_q_low,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Trend_timeperiod,Trend_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
306,Pattern_Trend,0.95,0.10,1,5,0.8,0.5,16,0,66,1.236867,0.529583,15.633200,34.952500,1.3468,1.4646,1.3953,1.2748,1.2489,1.1693,1.1767,1.2405,1.0583,1.0434,1.0259,1.0805,1.1309,1.2507,1.2289,1.2460,1.2444,1.2474,1.3061,1.3107,1.3023,1.3172,1.3168,1.2584,0.11,0.20,0.31,0.39,0.47,0.53,0.59,0.65,0.68,0.70,0.72,0.74,0.73,0.73,0.71,0.69,0.66,0.61,0.59,0.61,0.69,0.22,0.20,0.18
288,Pattern_Trend,0.90,0.10,1,5,0.8,0.5,16,0,71,1.270942,0.485833,19.236858,34.494167,1.4110,1.5131,1.4436,1.3177,1.2906,1.2321,1.2678,1.3273,1.1630,1.1416,1.1097,1.1325,1.1578,1.2672,1.2458,1.2613,1.2547,1.2417,1.2954,1.2999,1.2926,1.3056,1.3025,1.2281,0.10,0.19,0.29,0.37,0.44,0.50,0.57,0.63,0.66,0.67,0.69,0.70,0.68,0.67,0.65,0.63,0.60,0.54,0.53,0.54,0.61,0.16,0.14,0.10
308,Pattern_Trend,0.95,0.20,1,5,0.8,0.5,16,0,157,1.441812,0.434583,69.364562,68.229583,1.3364,1.4149,1.3816,1.2873,1.2947,1.5059,1.4650,1.5359,1.3935,1.3286,1.3027,1.3256,1.3390,1.3946,1.4416,1.4625,1.4987,1.5070,1.5464,1.5560,1.5272,1.5546,1.6060,1.5978,0.07,0.12,0.18,0.21,0.26,0.31,0.36,0.41,0.44,0.45,0.46,0.48,0.48,0.49,0.51,0.53,0.54,0.54,0.55,0.58,0.72,0.55,0.58,0.61
270,Pattern_Trend,0.85,0.10,1,5,0.8,0.5,16,0,74,1.273000,0.430000,20.202000,31.820000,1.2686,1.3835,1.4375,1.2897,1.2656,1.2335,1.2937,1.3543,1.1914,1.1691,1.1366,1.1572,1.1808,1.2866,1.2650,1.2813,1.2742,1.2606,1.3128,1.3167,1.3090,1.3216,1.3184,1.2443,0.09,0.16,0.26,0.33,0.40,0.46,0.52,0.58,0.60,0.61,0.62,0.63,0.61,0.60,0.58,0.57,0.54,0.48,0.46,0.46,0.53,0.10,0.08,0.05
290,Pattern_Trend,0.90,0.20,1,5,0.8,0.5,16,0,162,1.450100,0.416250,72.916200,67.432500,1.3630,1.4338,1.4003,1.3039,1.3103,1.5285,1.4998,1.5687,1.4348,1.3660,1.3336,1.3412,1.3433,1.3967,1.4416,1.4619,1.4944,1.4942,1.5322,1.5419,1.5146,1.5406,1.5887,1.5684,0.06,0.12,0.17,0.21,0.26,0.30,0.36,0.40,0.43,0.44,0.45,0.46,0.47,0.48,0.49,0.50,0.51,0.51,0.52,0.55,0.68,0.52,0.54,0.56
307,Pattern_Trend,0.95,0.15,1,5,0.8,0.5,16,0,112,1.326454,0.401667,36.562867,44.986667,1.3463,1.4757,1.3826,1.2768,1.2477,1.2314,1.2292,1.2664,1.1415,1.1074,1.0925,1.1545,1.2111,1.3035,1.3609,1.3841,1.4066,1.4385,1.4874,1.4768,1.4370,1.4573,1.4712,1.4485,0.07,0.13,0.19,0.23,0.27,0.30,0.35,0.39,0.41,0.42,0.44,0.46,0.47,0.48,0.49,0.50,0.51,0.50,0.52,0.55,0.66,0.41,0.44,0.45
252,Pattern_Trend,0.80,0.10,1,5,0.8,0.5,16,0,81,1.239167,0.397500,19.372500,32.197500,1.3799,1.4664,1.4739,1.3320,1.3020,1.2677,1.3155,1.3682,1.1812,1.1642,1.1074,1.1273,1.1534,1.2145,1.1923,1.1880,1.1591,1.1541,1.1974,1.2007,1.1975,1.2137,1.2178,1.1658,0.08,0.15,0.24,0.30,0.37,0.43,0.49,0.54,0.56,0.57,0.58,0.58,0.56,0.55,0.53,0.50,0.47,0.41,0.45,0.45,0.51,0.10,0.08,0.04
289,Pattern_Trend,0.90,0.15,1,5,0.8,0.5,16,0,117,1.341592,0.380000,39.966225,44.460000,1.3844,1.5024,1.4087,1.3002,1.2699,1.2658,1.2799,1.3144,1.2008,1.1626,1.1398,1.1824,1.2232,1.3109,1.3649,1.3869,1.4052,1.4244,1.4708,1.4613,1.4242,1.4430,1.4544,1.4177,0.07,0.13,0.18,0.22,0.26,0.30,0.34,0.38,0.41,0.42,0.43,0.45,0.45,0.46,0.46,0.47,0.47,0.46,0.48,0.51,0.62,0.37,0.39,0.39
272,Pattern_Trend,0.85,0.20,1,5,0.8,0.5,16,0,166,1.445450,0.373333,73.944700,61.973333,1.2981,1.3

In [4]:
# high/low bound
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg', ascending=False)
stat.head(20)

,pattern,Pattern_vol_q_high,Pattern_vol_q_low,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Trend_timeperiod,Trend_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
9,Pattern_Trend,0.9,0.2,3,5,0.8,0.5,16,0,103,1.290075,0.418750,29.877725,43.131250,1.2365,1.3188,1.2297,1.1964,1.2145,1.2410,1.2261,1.2686,1.1687,1.1469,1.1506,1.1422,1.1663,1.2506,1.2931,1.2879,1.3196,1.3465,1.4125,1.4188,1.4142,1.4697,1.5406,1.5020,0.07,0.12,0.18,0.22,0.27,0.34,0.40,0.44,0.47,0.48,0.47,0.48,0.46,0.46,0.46,0.47,0.48,0.49,0.51,0.54,0.75,0.47,0.50,0.52
10,Pattern_Trend,0.9,0.2,3,6,0.8,0.5,16,0,118,1.404954,0.402500,47.784592,47.495000,1.3342,1.3929,1.3385,1.3166,1.3217,1.3398,1.3295,1.3663,1.2714,1.2474,1.2517,1.2460,1.2941,1.3902,1.4265,1.4163,1.4512,1.4720,1.5441,1.5496,1.5428,1.5950,1.6608,1.6203,0.06,0.10,0.15,0.19,0.24,0.29,0.34,0.38,0.42,0.42,0.42,0.42,0.41,0.42,0.42,0.44,0.46,0.47,0.54,0.58,0.78,0.54,0.57,0.60
1,Pattern_Trend,0.9,0.2,1,5,0.8,0.5,16,0,161,1.339475,0.393750,54.655475,63.393750,1.3711,1.4290,1.3962,1.3094,1.3154,1.3131,1.3005,1.3742,1.2628,1.2071,1.1827,1.1979,1.2073,1.2642,1.3117,1.3363,1.3707,1.3752,1.4149,1.4278,1.4044,1.4310,1.4805,1.4640,0.06,0.12,0.17,0.21,0.26,0.30,0.35,0.40,0.43,0.43,0.44,0.45,0.45,0.45,0.46,0.47,0.48,0.47,0.48,0.50,0.63,0.46,0.48,0.50
8,Pattern_Trend,0.9,0.2,3,4,0.8,0.5,16,0,80,1.344688,0.392917,27.575000,31.433333,1.2943,1.4710,1.3352,1.3407,1.3376,1.3060,1.2581,1.3202,1.1676,1.1369,1.1888,1.2000,1.2375,1.3585,1.3883,1.3592,1.3805,1.3599,1.4274,1.4285,1.4196,1.4806,1.5596,1.5165,0.08,0.13,0.18,0.22,0.28,0.33,0.39,0.43,0.47,0.48,0.49,0.50,0.48,0.49,0.48,0.48,0.48,0.46,0.45,0.46,0.70,0.31,0.33,0.33
2,Pattern_Trend,0.9,0.2,1,6,0.8,0.5,16,0,176,1.412833,0.384583,72.658667,67.686667,1.4313,1.4735,1.4631,1.3853,1.3834,1.3760,1.3662,1.4354,1.3277,1.2706,1.2470,1.2628,1.2888,1.3535,1.3972,1.4187,1.4551,1.4564,1.5007,1.5124,1.4877,1.5131,1.5600,1.5421,0.06,0.10,0.15,0.19,0.23,0.28,0.32,0.36,0.39,0.40,0.40,0.42,0.42,0.43,0.44,0.45,0.46,0.46,0.50,0.53,0.66,0.50,0.53,0.55
5,Pattern_Trend,0.9,0.2,2,5,0.8,0.5,16,0,154,1.344550,0.381250,53.060700,58.712500,1.4262,1.4509,1.3953,1.3113,1.3088,1.3203,1.3094,1.3711,1.2574,1.1962,1.1715,1.1993,1.2040,1.2634,1.3133,1.3387,1.3755,1.3811,1.4222,1.4355,1.4115,1.4393,1.4923,1.4747,0.06,0.11,0.16,0.20,0.25,0.30,0.34,0.39,0.42,0.42,0.42,0.43,0.43,0.43,0.44,0.45,0.46,0.45,0.46,0.49,0.63,0.45,0.47,0.49
0,Pattern_Trend,0.9,0.2,1,4,0.8,0.5,16,0,139,1.375554,0.380833,52.202029,52.935833,1.4101,1.5263,1.4840,1.4080,1.4006,1.3527,1.3221,1.4036,1.2625,1.1968,1.1955,1.2268,1.2415,1.3119,1.3615,1.3801,1.4117,1.3999,1.4350,1.4456,1.4162,1.4432,1.4967,1.4810,0.06,0.12,0.17,0.21,0.26,0.29,0.34,0.39,0.41,0.43,0.44,0.46,0.46,0.47,0.48,0.48,0.48,0.46,0.45,0.46,0.60,0.39,0.41,0.42
6,Pattern_Trend,0.9,0.2,2,6,0.8,0.5,16,0,169,1.420475,0.373333,71.060275,63.093333,1.4873,1.4964,1.4648,1.3904,1.3796,1.3856,1.3774,1.4349,1.3249,1.2623,1.2385,1.2668,1.2888,1.3561,1.4019,1.4240,1.4627,1.4650,1.5107,1.5228,1.4974,1.5239,1.5741,1.5551,0.05,0.10,0.15,0.18,0.22,0.27,0.31,0.35,0.38,0.39,0.39,0.40,0.40,0.41,0.42,0.43,0.45,0.45,0.49,0.52,0.65,0.49,0.52,0.54
4,Pattern_Trend,0.9,0.2,2,4,0.8,0.5,16,0,132,1.383371,0.365833,50.604950,48.290000,1.4776,1.5588,1.4872,1.4158,1.3964,1.3637,

# Research parameter influence

In [3]:
param = 'Pattern_use_vol'

res = stat.groupby([param]).agg({'e_ratio_avg': 'mean',
                                 'pct_price_diff_avg': 'mean',
                                 'e_ratio_rank': 'mean', 
                                 'price_rank': 'mean', 
                                 'forecasts_num': 'sum'}).sort_values(param).reset_index()
res

,Pattern_use_vol,e_ratio_avg,pct_price_diff_avg,e_ratio_rank,price_rank,forecasts_num
0,-1,1.546973,2.873992,1061.440504,5577.185392,291081
1,0,1.387825,2.958303,865.087474,6598.829933,334592
2,1,1.387787,2.958333,865.153912,6600.041667,334650


# Plot result in file

In [23]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

x, y = param, 'e_ratio_avg'
ax.plot(res[x], res[y])
ax.set_xlabel(x)
ax.set_ylabel(y)
ax.set_title('_'.join(pattern) + '_' + ttype)

fig.savefig(f"{'_'.join(pattern)}'_'{ttype}_{param}", dpi=fig.dpi)

# Save new config data to config file

In [5]:
from config_updater import ConfigUpdater

ttype = 'buy'
pattern = ['Pattern', 'Trend']
work_timeframe = '15m'
higher_timeframe = '1h'
timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {
                 'Pattern': {
                            'vol_q_high': [0.9],
                            'vol_q_low': [0.2],
                            'window_low_bound': [1], 
                            'window_high_bound': [6],
                            'first_candle': [0.8],
                            'second_candle': [0.5],
                            },
                'Trend': {
                            'timeperiod': [16], 
                            'low_bound': [0]
                            }
             }
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)